In [1]:
from ATARI.theory.resonance_statistics import make_res_par_avg
from ATARI.theory.scattering_params import FofE_recursive
import ATARI.utils.hdf5 as h5io
from matplotlib.pyplot import *
import numpy as np
from scipy.linalg import svdvals
import pandas as pd
import importlib
import os
from copy import copy
from ATARI.sammy_interface import sammy_classes, sammy_functions, template_creator
from ATARI.sammy_interface.sammy_deriv import get_derivatives, find_interpolation_array, interpolate_derivatives

from ATARI.ModelData.particle import Particle, Neutron
from ATARI.ModelData.particle_pair import Particle_Pair
from ATARI.ModelData.experimental_model import Experimental_Model

from ATARI.Bayes.bayes_update import Bayes

In [2]:
sammypath = ''
assert(sammypath != '')

In [3]:
%matplotlib widget

# ATARI Sammy Interface

This user example details basic use of the ATARI/SAMMY interface module. 
The example given here shows how to do individual sammy runs using the NV or IQ solution scheme.
The AutoFit example will detail sammy interface with YW scheme that can be used for automatic evaluations or simultaneous data fitting.

In [ ]:
### setup the reaction model and sample a resonance
Ta181 = Particle(Z=73, A=181, I=3.5, mass=180.94803, name='Ta181')
Ta_pair = Particle_Pair(isotope = "Ta181",
                        resonance_ladder = pd.DataFrame(),
                        formalism = "XCT",
                        energy_range = [175,225],
                        ac = 0.8127,
                        target=Ta181,
                        projectile=Neutron,
                        l_max = 1)
Ta_pair.add_spin_group(Jpi='3.0',
                       J_ID=1,
                       D=9.0030,
                       gn2_avg=452.56615,
                       gn2_dof=1,
                       gg2_avg=32.0,
                       gg2_dof=1000)
Ta_pair.resonance_ladder = pd.DataFrame({'E':[197.0, 203.0], 'Gg':[64.0, 65.0], 'Gn1':[22.7266399211, 32.0], 'J_ID':[1.0, 1.0], 'Jpi':[3.0, 3.0], 'L':[0, 0]})
Ta_pair.expand_ladder()
Ta_pair.resonance_ladder[["varyE", "varyGg", "varyGn1"]] = 1
# Ta_pair.resonance_ladder[["varyE", "varyGg", "varyGn1"]] = 0
# Ta_pair.resonance_ladder[["varyGn1"]] = 1
# Ta_pair.resonance_ladder[["varyGg"]] = 1
# Ta_pair.resonance_ladder[["varyE"]] = 1

# setup experimental transmission model
# exp_model_T = Experimental_Model(energy_range=Ta_pair.energy_range, energy_grid=[199,201])
exp_model_T = Experimental_Model(energy_range=Ta_pair.energy_range)


# calculate experimentally corrected transmission or capture yield with sammy
rto = sammy_classes.SammyRunTimeOptions(sammypath,
                                        Print             = True,
                                        bayes             = True,
                                        derivatives       = False,
                                        bayes_scheme      = 'NV',
                                        iterations        = 1,
                                        use_least_squares = True,
                                        keep_runDIR       = True,
                                        sammy_runDIR      = 'sammy_runDIR_2')

template_creator.make_input_template('template_T.inp', Ta_pair, exp_model_T, rto)
exp_model_T.template = os.path.realpath('template_T.inp')

Ta_pair.resonance_ladder

In [ ]:
### Generate syndat from measurement models
from ATARI.ModelData.measurement_models.transmission_rpi import Transmission_RPI
from ATARI.syndat.control import Syndat_Model, syndatOPT


synOPT = syndatOPT(sampleRES=False, calculate_covariance=True, explicit_covariance=True, )

synT = Syndat_Model(generative_experimental_model=exp_model_T, options=synOPT)

## need to test syndat covariance generation with different tof ordering !!!

synT.sample(Ta_pair, 
            sammyRTO=rto,
            num_samples=1)

datasample = synT.samples[0]
print(datasample.pw_reduced)
data = datasample.pw_reduced
# data['exp'] = data['true']
# data['exp_unc'] = 0.000001
# data['exp_unc'] = 0.00001

data

In [6]:
# # Making a simple dataset
# data = pd.DataFrame({'tof': ['dunno'], 'E':[200.0], 'true':[0.6], 'exp':[0.6], 'exp_unc':[0.01]})

In [ ]:
# print(vars(generative.model_parameters))
Ta_pair.resonance_ladder

In [ ]:
init_par_uncert = 1.0

# Altering a bad ladder:
res_ladder_bad = copy(Ta_pair.resonance_ladder)

# print(data.exp.values)
exp_data = pd.DataFrame({'E':data.E, 'exp_trans':data.exp, 'exp_unc':data.exp_unc})
sammyINP = sammy_classes.SammyInputData(Ta_pair,
                                        res_ladder_bad,
                                        os.path.realpath('template_T.inp'),
                                        exp_model_T,
                                        experimental_data = exp_data,
                                        energy_grid = exp_model_T.energy_grid,
                                        initial_parameter_uncertainty=init_par_uncert)

print('Prior')
print(res_ladder_bad[['E', 'Gg', 'Gn1']])
print()

sammy_out = get_derivatives(sammyINP, rto, find_theo_trans=True, u_or_p='u')
derivatives = sammy_out.derivatives
par_post, Mp = Bayes(sammy_out, Ta_pair, rto, init_par_uncert)
print('Custom Bayes:')
print(par_post[['E', 'Gg', 'Gn1']])
print()

rto_bayes = sammy_classes.SammyRunTimeOptions(rto.path_to_SAMMY_exe,
                                              derivatives = False,
                                              bayes_scheme = 'NV')
sammy_out = sammy_functions.run_sammy(sammyINP, rto)
print('SAMMY Bayes:')
print(sammy_out.par_post[['E', 'Gg', 'Gn1']])
print()

In [ ]:
raise RuntimeError()

# Calculate Derivatives

In [ ]:
l = 0
Jpi = 3.0
gg20 = list(Ta_pair.spin_groups.values())[0]['<gg2>']
gn20 = list(Ta_pair.spin_groups.values())[0]['<gn2>']
Els = np.linspace(50, 10_000, 100)
Ggs = np.array([2*gg20])
Gns = np.linspace(100, 800, 5)
Ls = np.array([0])
Es = np.linspace(-10, 10, 100)
points, derivatives = find_interpolation_array(Ta_pair, exp_model_T, rto, Els, Ggs, Gns, Ls, Es, u_or_p='u')

interpolate_derivatives(points, derivatives, ...)

dP_dE_flat = dP_dE.reshape(num_energies,-1)
svs = svdvals(dP_dE_flat)
svs = np.sort(svs)[::-1]

figure(420)
clf()
plot(range(len(svs)), abs(svs), '.k')
yscale('log')
grid()
xlabel('Rank', fontsize=16)
ylabel('Singular Value', fontsize=16)
show()

In [ ]:
# l = 0
# Jpi = 3.0
# gg20 = list(Ta_pair.spin_groups.values())[0]['<gg2>']
# # gn20 = list(Ta_pair.spin_groups.values())[0]['<gn2>']
# num_energies = 100
# Es = np.linspace(1e-5, 10_000, 100)
# Gns = np.linspace(100, 800, 5)
# dP_dE = np.zeros((num_energies, len(Es), len(Gns), 3))
# for i, E0 in enumerate(Es):
#     print(f'El = {E0}')
#     for j, Gn0 in enumerate(Gns):
#         E = np.array([E0])
#         EB = (E[0]-10,E[0]+10)
#         Ta_pair.energy_range = EB
#         exp_model_T.energy_range = EB
#         E_grid = np.linspace(*EB, num_energies+2)[1:-1]
#         P = FofE_recursive(E, Ta_pair.ac, Ta_pair.M, Ta_pair.m, l)[1][l,:]
#         Gg  = 2*gg20 * np.ones((1,))
#         Gn  = Gn0 * np.ones((1,))
#         gg2 = gg20 * np.ones((1,))
#         gn2 = Gn0/(2*P) * np.ones((1,))
#         Ta_pair.resonance_ladder = pd.DataFrame([E, Gg, Gn, [1], gg2, gn2, [Jpi], [l]],
#                                             index=['E', 'Gg', 'Gn1',  'J_ID', 'gg2', 'gn2', 'Jpi', 'L',]).T
#         exp_model_T.energy_grid = E_grid
#         sammyINP = sammy_classes.SammyInputData(
#             Ta_pair,
#             Ta_pair.resonance_ladder,
#             os.path.realpath('template_T.inp'),
#             exp_model_T,
#             energy_grid=exp_model_T.energy_grid
#         )
#         sammy_out = get_derivatives(sammyINP, rto)
#         dP_dE[:,i,j,:] = sammy_out.derivatives

# dP_dE_flat = dP_dE.reshape(num_energies,-1)
# svs = svdvals(dP_dE_flat)
# svs = np.sort(svs)[::-1]

# figure(420)
# clf()
# plot(range(len(svs)), abs(svs), '.k')
# yscale('log')
# grid()
# xlabel('Rank', fontsize=16)
# ylabel('Singular Value', fontsize=16)
# show()

In [ ]:
dP_dE_flat = dP_dE.reshape(num_energies,-1)
svs = svdvals(dP_dE_flat)
svs = np.sort(svs)[::-1]

figure(421)
clf()
plot(range(len(svs)), abs(svs), '.k')
yscale('log')
grid()
xlabel('Rank', fontsize=16)
ylabel('Singular Value', fontsize=16)
show()

In [ ]:
# l = 0
# Jpi = 3.0
# gg20 = 1.0
# gn20 = 1.0
# EB = (5,500)
# E = np.arange(*EB, 10)
# N_res = len(E)

# Ta_pair.energy_range = EB
# exp_model_T.energy_range = EB

# E_grid = np.linspace(*EB, 10_000)
# P = FofE_recursive(E, Ta_pair.ac, Ta_pair.M, Ta_pair.m, l)[1][l,:]
# Gg = 2*gg20 * np.ones((N_res,))
# Gn = 2*P*gn20 * np.ones((N_res,))
# gg2 = gg20 * np.ones((N_res,))
# gn2 = gn20 * np.ones((N_res,))
# Ta_pair.resonance_ladder = pd.DataFrame([E, Gg, Gn, [1]*N_res, gg2, gn2, [Jpi]*N_res, [l]*N_res],
#                                      index=['E', 'Gg', 'Gn1',  'J_ID', 'gg2', 'gn2', 'Jpi', 'L',]).T
# exp_model_T.energy_grid = E_grid
# sammyINP = sammy_classes.SammyInputData(
#     Ta_pair,
#     Ta_pair.resonance_ladder,
#     os.path.realpath('template_T.inp'),
#     exp_model_T,
#     energy_grid=exp_model_T.energy_grid
# )
# sammy_out = get_derivatives(sammyINP, rto)
# print(sammy_out.derivatives)

In [ ]:
raise RuntimeError('STOP!!!')

In [ ]:
### Option to read in idc or you can just pass to sammy a filepath


# def read_idc(filepath):
#     data = {
#         'J': {},
#         'C': None,
#         'stat': None
#     }

#     with open(filepath, 'r') as f:
#         num_params = None
#         in_partial_derivatives = False
#         in_uncertainties = False
#         in_correlations = False
#         for line in f.readlines():
            
#             if line.lower().startswith("nu"):
#                 num_params = int(line.split()[-1])
            
#             elif line.lower().startswith("free-forma"):
#                 in_partial_derivatives = True

#             elif line.lower().startswith("uncertaint"):
#                 in_partial_derivatives = False
#                 in_uncertainties = True
            
#             elif line.lower().startswith("correlatio"):
#                 in_uncertainties = False
#                 in_correlations = True

#             elif in_partial_derivatives:
#                 splitline = line.split()
#                 E = float(splitline[0])
#                 stat_unc = float(splitline[1])
#                 derivatives = [float(x) for x in splitline[2:]]
#                 data['J'][E] = {'stat_unc': stat_unc, 'derivatives': derivatives}
                
#             elif in_uncertainties:
#                 uncertainties = [float(e) for e in line.split()]
#                 data['C'] = np.diag(uncertainties)

#             elif in_correlations:
#                 assert isinstance(num_params, int)
#                 correlations = []
#                 for _ in range(num_params):
#                     line = f.readline().strip().split()
#                     correlations.append([float(x) for x in line])

#     data['stat'] = None  # You need to fill in the logic for reading the 'stat' data

#     return data

# Usage
# filepath = '/Users/noahwalton/research_local/resonance_fitting/ATARI_workspace/measurement_data/trans-Ta-1mm.idc'
# read_data = read_idc(filepath)

In [ ]:
print(datasample.covariance_data.keys())

In [ ]:
datasample.covariance_data['CovT']

In [ ]:
# ### decomposed covariance test
# stat = copy(datasample.covariance_data["diag_stat"])
# CT = copy(datasample.covariance_data['CovT'])
# J = copy(datasample.covariance_data['Jac_sys'])
# C = copy(datasample.covariance_data['Cov_sys'])
# # C = np.diag(np.diag(C))
# test = J.T@C@J
# test.index.name = None
# assert(np.max(abs((np.diag(stat.var_stat) + test) - CT)) == 0.0)

# Fit the data with sammy

In [ ]:
rto.bayes=True
rto.get_ECSCM = True
rto.ECSCM_rxn = 'transmission'

sammyINP = sammy_classes.SammyInputData(
    Ta_pair,
    Ta_pair.resonance_ladder,
    os.path.realpath('template_T.inp'),
    exp_model_T,
    # energy_grid=exp_model_T.energy_grid
    experimental_data=data,
    experimental_covariance = datasample.covariance_data
)

sammyINP.initial_parameter_uncertainty=10

# std = 0.01
# data.exp = np.random.default_rng().normal(data.true, std)
# data.exp_unc = std

sammyINP.experimental_data = data
sammyINP.resonance_ladder["varyE"] = np.ones(len(Ta_pair.resonance_ladder))
sammyINP.resonance_ladder["varyGg"] = np.ones(len(Ta_pair.resonance_ladder))
sammyINP.resonance_ladder["varyGn1"] = np.ones(len(Ta_pair.resonance_ladder))

sammyOUT2 = sammy_functions.run_sammy(sammyINP, rto)
print(sammyOUT2.chi2_post)
print(sammyOUT2.chi2n_post)

BACKGround functions
EXPON 0 0 582.77685 33.822441 0.0514968 0.0046811 

NORMALIZATION AND BACKGROUND ARE NEXT
1.0000000        0.0                                         3
0.0384200

!! when fitting background or normalization, the output lst has an additional column I need to be robust to.

In [ ]:

### Plot 
figure()

errorbar(data.E, data.exp, yerr=data.exp_unc, zorder=0,
                                        fmt='.', color='k', linewidth=0.5, markersize=1.5, capsize=1, label='12mm')

plot(data.E, data.true, 'g')

plot(sammyOUT2.pw.E, sammyOUT2.pw.theo_trans, 'b')
plot(sammyOUT2.pw_post.E, sammyOUT2.pw_post.theo_trans_bayes, 'r')

# plot(sammyOUT2.pw.E, sammyOUT2.pw.theo_xs, 'b')
# plot(sammyOUT2.pw_post.E, sammyOUT2.pw_post.theo_xs_bayes, 'r')
# # plot(sammyOUT_old.est_df.E, sammyOUT_old.est_df.theo, 'b')
# # sammyOUT_old = copy(sammyOUT2)

x = sammyOUT2.est_df.E
y = sammyOUT2.est_df.theo
y_err=  sammyOUT2.est_df.theo_unc #
# y_err = np.sqrt(np.diag(sammyOUT2.ECSCM))
fill_between(x, y - y_err, y + y_err, color='r', alpha=0.5, label='Error Band')
plot(x, y, 'r')

ylabel("T")

# xlim([200,225])
# ylim([-0.1,1.1])
legend()

xlabel('Energy (eV)')
tight_layout()


# figure()
# imshow(sammyOUT2.ECSCM)
# colorbar()

In [ ]:
ladder = copy(sammyOUT2.par_post)
print(ladder)
Ta_pair.spin_groups

In [ ]:
from ATARI.utils.atario import expand_sammy_ladder_2_atari

expand_sammy_ladder_2_atari(Ta_pair, ladder)
ladder

In [ ]:
# samples = 100

# cov_true = np.zeros([len(sammyOUT2.pw),len(sammyOUT2.pw)])
# cov_est = np.zeros([len(sammyOUT2.est_df),len(sammyOUT2.est_df)])

# for i in range(samples):
#     # synT.run(sammyOUT.pw)
#     data.exp = np.random.default_rng().normal(synT.data.true, std)
#     data.exp_unc = std
#     sammyINP.experimental_data = synT.data
#     sammyOUT2 = sammy_functions.run_sammy(sammyINP, rto)
#     residual = np.atleast_2d(sammyOUT2.pw.theo_trans_bayes) - np.atleast_2d(synT.data.true)
#     cov_true += residual.T@residual
#     cov_est += sammyOUT2.ECSCM
#     # true.append(cov_true)
#     # est.append(cov_est)

In [ ]:
# iest = 0
# fig, axes = subplots(1,2, figsize=(10,4))
# # im1 = axes[0].imshow(np.log10(cov_true/(samples)))
# # im2 = axes[1].imshow(np.log10(cov_est/(samples-1)))
# im1 = axes[0].pcolormesh(cov_true/(samples), clim=(-1e-5, 8e-5))
# im2 = axes[1].pcolormesh(cov_est/(samples-1), clim=(-1e-5, 8e-5))
# axes[0].set_title("empirical")

# axes[1].set_title("estimated")
# # for ax in axes:
# colorbar(im1)

# colorbar(im2)

# print("Empirical Fnorm")
# print(np.linalg.norm(cov_true/(samples), ord='fro'))
# print("Estimated Fnorm")
# print(np.linalg.norm(cov_est/(samples-1), ord='fro'))

In [ ]:
# ### Plot 
# figure()

# # errorbar(synT.data.E, synT.data.exp, yerr=synT.data.exp_unc, zorder=0,
# #                                         fmt='.', color='k', linewidth=0.5, markersize=1.5, capsize=1, label='12mm')

# # plot(synT.data.E, synT.data.true, 'g')
# plot(synT.data.E, np.sqrt(np.diag(cov_true/samples)), label="empirical")
# plot(x, np.sqrt(np.diag(cov_est/(samples-1))), label="mean estimated")


# xlim([200,225])
# # ylim([-0.1,1.1])
# legend()

# xlabel('Energy (eV)')
# tight_layout()

In [ ]:
# # result_dict = {}
# # stds = [0.1, 0.01, 0.001, 0.0001]
# stds = [0.00001, 0.05]

# for istd in stds:
#     samples = 1000
#     cov_true = np.zeros([len(sammyOUT2.pw), len(sammyOUT2.pw)])
#     cov_est = np.zeros([len(sammyOUT2.est_df), len(sammyOUT2.est_df)])
#     for i in range(samples):
#         # synT.run(sammyOUT.pw)
#         synT.data.exp = np.random.default_rng().normal(synT.data.true, istd)
#         synT.data.exp_unc = istd
#         sammyINP.experimental_data = synT.data
#         sammyOUT2 = sammy_functions.run_sammy(sammyINP, rto)
#         residual = np.atleast_2d(
#             sammyOUT2.pw.theo_trans_bayes) - np.atleast_2d(synT.data.true)
#         cov_true += residual.T@residual
#         cov_est += sammyOUT2.ECSCM
        
#     result_dict[istd] = [cov_true, cov_est]

In [ ]:
# stds = [1.0, 0.1, 0.05, 0.01, 0.001, 0.0001]
# true= []
# est = []

# for istd in stds:
#     res = result_dict[istd]
#     cov_true = res[0]
#     cov_est = res[1]
#     print(istd)
#     # print("Empirical Fnorm: ", np.linalg.norm(cov_true/(samples), ord='fro'))
#     # print("Estimated Fnorm: ", np.linalg.norm(cov_est/(samples-1), ord='fro'))
#     # true.append(np.linalg.norm(cov_true/(samples), ord='fro'))
#     # est.append(np.linalg.norm(cov_est/(samples-1), ord='fro'))
#     print("Empirical Fnorm: ", np.sum(np.diag(cov_true)**2/(samples)))
#     print("Estimated Fnorm: ", np.sum(np.diag(cov_est)**2/(samples-1)))
#     true.append(np.sum(np.diag(cov_true)**2/(samples)))
#     est.append(np.sum(np.diag(cov_est)**2/(samples-1)))

In [ ]:
# figure()
# plot(stds, true, '.', label='Empirical')
# plot(stds, est, '.r', label='Estimate')
# xscale("log")
# yscale("log")
# legend()
# # ylabel("Noise Level")
# xlabel("Noise Level")